In [1]:
def ext_scalars(k, n):
    q = k.order()
    assert gcd(n, q) == 1, "Only multiplicative H90, please!"
    
    phi = cyclotomic_polynomial(n, 'z').change_ring(k)
    
    K = k.extension(n, 'x')
    A = K['z']
    h = A(phi.factor()[0][0])
    
    return K, A.quo(h, 'z')

In [2]:
def frob_l(a, i=1):
    A = a.parent()
    q = a.base_ring().base_ring().order()
    return A(list(c^(q^i) for c in a))

In [3]:
def frob_r(a, i=1):
    A = a.parent()
    z = A.gen()
    q = a.base_ring().base_ring().order()
    return sum(c*z^(j*q^i) for j, c in enumerate(a))

In [4]:
def is_h90(a, zeta):
    return frob_l(a) == zeta * a

In [5]:
def solve_h90(A, z = None):
    x = A(1)
    n = A.base_ring().degree()
    if z == None:
        z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        x = sum(frob_l(x, i)*z^-i for i in range(n))
    return x

In [6]:
def cyclo_deg(k, n):
    return Zmod(n)(k.characteristic()).multiplicative_order()

def rnorm(a, m, n = 0):
    A = a.parent()
    if n == 0:
        n = A.base_ring().degree()
    q = A.base_ring().base_ring().order()
    s = cyclo_deg(GF(q), m)
    d = n // m
    
    return product(frob_r(a, s*j) for j in range(d))

def toto(k, m, n):
    q = k.order()
    r = cyclo_deg(k, m)
    s = cyclo_deg(k, n)
    t = (q^s-1)//(q^r-1)
    return t, gcd(t, n), n//m

def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]

def testl(p, l, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, l^n)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(l^(n+1))
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(l^(n+1)))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, l^n)
    aa = h1^(l^n)
    bb = i2^(l^n)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(l^(n+1))
    return h1, A2(c.polynomial().list())*h2

def is_iso(h1, h2):
    m = h1.base_ring().degree()
    n = h2.base_ring().degree()
    l = n // m
    i2 = rnorm(h2, m)
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return h1.list()[0].minpoly() == j2.minpoly()  
    
def frob_rc(a, n, m, i=1):
    A = a.parent()
    z = A.gen()
    l = n // m
    u, v = xgcd(l, m)[1:]
    q = a.base_ring().base_ring().order()
    return sum(c*z^(u*l*j)*z^(v*m*j*q^i) for j, c in enumerate(a))

def rnormc(a, m):
    A = a.parent()
    n = a.base_ring().degree()
    q = A.base_ring().base_ring().order()
    l = n // m
    d = cyclo_deg(GF(q), l)
    print d, l
    return product(frob_rc(a, n, m, j) for j in range(d))

"""
Here we find an element h2 solution of H90 in F_p^n compatible with h1.
"""
def findh90(h1, n):
    k = h1.base_ring().base_ring()
    k1, A1 = h1.base_ring(), h1.parent()
    M1 = A1.modulus()
    R = k['z']
    m = k1.degree()
    l = n // m
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(M1.change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^(m)
    bb = i2^(m)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return A2(c.polynomial().list())*h2

"""
Return two elements h1, h2 that are respectively solutions of H90 in 
A_m = F_p^m ⊗ F_p(ζ_m) for the root 1 ⊗ ζ_m and in A_n for 1 ⊗ ζ_n, such that
N(h2) = h1.
"""
def testc(p, m, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, m)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    l = n // m
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^m
    bb = i2^m
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return h1, A2(c.polynomial().list())*h2

def conv(x):
    z = x.parent().gen()
    M = z.minpoly()
    k = x.base_ring().base_ring()
    C = k.extension(M, 'z')
    return C([k(s) for s in x.list()])

# Fonctions archives

In [7]:
def basis_matrix(a, n = None):
    K = a.parent()
    m = K.degree()
    
    if n == None:
        n = m
    
    k = K.prime_subfield()
    S = MatrixSpace(k, m, n)()
    for j in range(n):
        L = (a^j).polynomial().list()
        i = 0
        for l in L:
            S[i, j] = l
            i += 1
    return S

def fflist(x):
    L = x.polynomial().list()
    l = len(L)
    k = x.parent()
    d = k.degree()
    if l < d:
        L += (d-l)*[k()]
        
    return L

def compute_map(a, b):
    
    A = basis_matrix(a)
    B = basis_matrix(b, a.parent().degree())
    C = B*A^(-1)
    
    K = b.parent()
    k = K.prime_subfield()
    S = MatrixSpace(k, a.parent().degree(), 1)

    return lambda x : K((C*S(fflist(x))).column(0))

def compute_emb(h1, h2):
    m = h1.base_ring().degree()
    n = h2.base_ring().degree()
    l = n // m
    i2 = rnorm(h2, m)
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return compute_map(h1.list()[0], j2) 

In [8]:
"""
Test function to work around the extension F_{p^l}, F_{p^{lm}}, F_{p^{l²m²}}.
"""
def test_comp(p, l, m):
    
    # We create the prime field, the root ζ_{l²m²}, and the field k(ζ_{l²m²})
    
    k = GF(p)
    R = k['z']
    Cyclo = cyclotomic_polynomial(l^2*m^2)
    Zl2m2 = Cyclo.change_ring(k).factor()[0][0]
    Cl2m2.<z> = k.extension(Zl2m2)
    
    # We create the other roots using ζ_{l²m²} (we are cheating a bit...)
    
    zl2 = z^(m^2)
    zm2 = z^(l^2)
    Zl2 = zl2.minpoly()
    Zm2 = zm2.minpoly()
    
    zl = zl2^l
    zm = zm2^m
    Zl = zl.minpoly()
    Zm = zm.minpoly()
    
    u1, u2 = xgcd(l, m)[1:3]
    zlm = zl^u2 * zm^u1
    Zlm = zlm.minpoly()
    
    # We create the rings and algebras
    
    kl2m2 = k.extension(l^2*m^2, "x")
    Rl2m2 = kl2m2["T"]
    Al2m2 = Rl2m2.quo(Zl2m2)
    
    kl2 = k.extension(l^2, "x")
    Rl2 = kl2["T"]
    Al2 = Rl2.quo(Zl2)

    km2 = k.extension(m^2, "x")
    Rm2 = km2["T"]
    Am2 = Rm2.quo(Zm2)
    
    klm = k.extension(l*m, "x")
    Rlm = klm["T"]
    Alm = Rlm.quo(Zlm)
    
    kl = k.extension(l, "x")
    Rl = kl["T"]
    Al = Rl.quo(Zl)
    
    km = k.extension(m, "x")
    Rm = km["T"]
    Am = Rm.quo(Zm)
    
    # We compute the solutions of the Hilbert 90 problems in the first extension of each tower
    
    hl = solve_h90(Al)
    hm = solve_h90(Am)
    
    # We compute compatible solutions in higher extensions of the two towers
    ## l-adic tower
    
    htmpl2 = solve_h90(Al2)
    il2 = rnorm(htmpl2, l)
    Cl2 = k.extension(Zl2, 'x')
    aal = hl^(l)
    bbl = il2^(l)
    al = R([x for x in aal.list()]).subs(Cl2.gen()^l)
    bl = Cl2([k(x) for x in bbl.list()])
    
    tl = al/bl
    cl = tl.nth_root(l^2)
    hl2 = Al2(cl.polynomial().list())*htmpl2
    
    ## m-adic tower
    
    htmpm2 = solve_h90(Am2)
    im2 = rnorm(htmpm2, m)
    Cm2 = k.extension(Zm2, 'x')
    aam = hm^(m)
    bbm = im2^(m)
    am = R([x for x in aam.list()]).subs(Cm2.gen()^m)
    bm = Cm2([k(x) for x in bbm.list()])
    
    tm = am/bm
    cm = tm.nth_root(m^2)
    hm2 = Am2(cm.polynomial().list())*htmpm2
    
    # We compute compatible solutions in the composita
    ## l × m
    
    Clm = k.extension(Zlm, 'x')
    htmplm = solve_h90(Alm)
    alm = Clm([k(x) for x in (htmplm^(l*m)).list()])
    blm = Clm([k(x) for x in (hl^l).list()]).polynomial().subs(Clm.gen()^m)
    clm = Clm([k(x) for x in (hm^m).list()]).polynomial().subs(Clm.gen()^l)
    
    tl_lm = (blm*alm^(-1)).nth_root(l)
    tm_lm = (clm*alm^(-1)).nth_root(m)
    
    HL = Alm(tl_lm.polynomial().list())*htmplm^m
    HM = Alm(tm_lm.polynomial().list())*htmplm^l
    
    hlm = (HL^u2)*(HM^u1)
    
    Tlm_m = lambda x : Alm(((clm/blm)^u2).polynomial().list())*x^l
    Tlm_l = lambda x : Alm(((blm/clm)^u1).polynomial().list())*x^m
    
    ## l² × m²
    
    v1, v2 = xgcd(l^2, m^2)[1:3]
    Cl2m2 = k.extension(Zl2m2, 'x')
    
    htmpl2m2 = solve_h90(Al2m2) #!# 2/3 of the time here
    
    al2m2 = Cl2m2([k(x) for x in (htmpl2m2^(l^2*m^2)).list()])
    bl2m2 = Cl2m2([k(x) for x in (hl2^(l^2)).list()]).polynomial().subs(Cl2m2.gen()^(m^2))
    cl2m2 = Cl2m2([k(x) for x in (hm2^(m^2)).list()]).polynomial().subs(Cl2m2.gen()^(l^2))
    
    tl2_l2m2 = (bl2m2*al2m2^(-1)).nth_root(l^2) 
    tm2_l2m2 = (cl2m2*al2m2^(-1)).nth_root(m^2)
    
    HL2 = Al2m2(tl2_l2m2.polynomial().list())*htmpl2m2^(m^2)
    HM2 = Al2m2(tm2_l2m2.polynomial().list())*htmpl2m2^(l^2)
    
    hl2m2 = (HL2^v2)*(HM2^v1)
    
    BLM = Cl2m2([k(x) for x in (hl^l).list()]).polynomial().subs(Cl2m2.gen()^(l*m^2))
    CLM = Cl2m2([k(x) for x in (hm^m).list()]).polynomial().subs(Cl2m2.gen()^(m*l^2))
    
    TLM_M = lambda x : Al2m2(((CLM/BLM)^u2).polynomial().list())*x^l
    TLM_L = lambda x : Al2m2(((BLM/CLM)^u1).polynomial().list())*x^m
    
    Tl2m2_m2 = lambda x : Al2m2(((cl2m2/bl2m2)^v2).polynomial().list())*x^(l^2)
    Tl2m2_l2 = lambda x : Al2m2(((bl2m2/cl2m2)^v1).polynomial().list())*x^(m^2)
    

    E1 = rnorm(Tl2m2_l2(hl2m2), l, l^2) # ~ hl #!# (a)
    E2 = rnorm(Tl2m2_m2(hl2m2), m, m^2) # ~ hm #!# (a) + here: 1/3 of the time
    
    E3 = (E1^u2)*(E2^u1) # ~ hlm
    E4 = TLM_L(E3) # ~ hl
    
    return hl2m2, E3
    
    # WE HAVE E1 == E4 meaning that T_{l²m²/l} = T_{lm/l} ∘ T_{l²m²/lm}

    h = compute_map(hl.list()[0], change_basis(E1, E1.parent().gen()^(l*m^2), hl.parent().degree()))
    f = compute_map(hl.list()[0], change_basis(Tlm_l(hlm), parent(hlm).gen()^(m), hl.parent().degree()))
    g = compute_map(hlm.list()[0], change_basis(E3, E3.parent().gen()^(l*m), hlm.parent().degree()))
    
    return g(f(kl.gen())) == h(kl.gen())

In [9]:
def trace_rel(a, m):
    k = a.base_ring().base_ring()
    p = k.characteristic()
    s = cyclo_deg(k, m)
    n = a.parent().degree()
    return sum(frob_r(a, s*j) for j in range(n/s))

def test_trace(p, l, m):

    # We create the prime field, the root ζ_{l²m²}, and the field k(ζ_{l²m²})
    
    k = GF(p)
    R = k['z']
    Cyclo = cyclotomic_polynomial(l*m)
    Zlm = Cyclo.change_ring(k).factor()[0][0]
    Clm.<z> = k.extension(Zlm)
    
    # We create the other roots using ζ_{lm} (we are cheating a bit...)
    
    zl = z^(m)
    zm = z^(l)
    Zl = zl.minpoly()
    Zm = zm.minpoly()
    
    # We create the rings and algebras
    
    klm = k.extension(l*m, "x")
    Rlm = klm["T"]
    Alm = Rlm.quo(Zlm)
    
    kl = k.extension(l, "x")
    Rl = kl["T"]
    Al = Rl.quo(Zl)
    
    km = k.extension(m, "x")
    Rm = km["T"]
    Am = Rm.quo(Zm)
    
    # We compute solutions of Hilbert 90 problem
    
    hl = solve_h90(Al)
    hm = solve_h90(Am)
    
    glm = solve_h90(Alm)
    
    # We look at what happens with traces
    
    gl = trace_rel(glm^m, l)
    gm = trace_rel(glm^l, m)
    
    # We find the constants between hl/hm and gl/gm
    
    GL = Clm([k(x) for x in (gl^(l)).list()])
    HL = Clm([k(x) for x in (hl^l).list()]).polynomial().subs(Clm.gen()^m)
    
    cl = (HL/GL).nth_root(l)

    GM = Clm([k(x) for x in (gm^(m)).list()])
    HM = Clm([k(x) for x in (hm^m).list()]).polynomial().subs(Clm.gen()^l)
    
    cm = (HM/GM).nth_root(m)
    
    return cl, cm

In [10]:
def test_trace2(p, l, n):

    k = GF(p)
    k1, A1 = ext_scalars(k, l^n)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(l^(n+1))
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(l^(n+1)))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = trace_rel(h2^l, l^n)
    aa = h1^(l^n)
    bb = i2^(l^n)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(l^(n+1))
    return h1, A2(c.polynomial().list())*h2

In [7]:
def rnorm2(a, m, n = 0):
    A = a.parent()
    if n == 0:
        n = A.base_ring().degree()
    q = A.base_ring().base_ring().order()
    t = cyclo_deg(GF(q), n)
    s = cyclo_deg(GF(q), m)
    d = t // s
    
    return product(frob_r(a, s*j) for j in range(d))

In [30]:
"""
Test function to work around the extension F_{p^l}, F_{p^{m}}, F_{p^{lm}}.
"""
def test_comp_norm(p, l, m):
    
    # We create the prime field, the root ζ_{lm}, and the field k(ζ_{lm})
    
    k = GF(p)
    R = k['z']
    Cyclo = cyclotomic_polynomial(l*m)
    Z = Cyclo.change_ring(k).factor()[0][0]
    Clm.<z> = k.extension(Z)
    d = Z.degree()
    
    # We create the other roots using ζ_{lm} (we are cheating a bit...)
    
    zl = z^(m)
    zm = z^(l)
    Zl = zl.minpoly()
    Zm = zm.minpoly()
    
    a = inverse_mod(d//cyclo_deg(GF(p),l),l)
    b = inverse_mod(d//cyclo_deg(GF(p),m),m)
    
    zlm = zl^a * zm^b
    Zlm = zlm.minpoly()
    
    # We create the rings and algebras
    
    klm = k.extension(l*m, "x")
    Rlm = klm["T"]
    Alm = Rlm.quo(Z)
    
    kl = k.extension(l, "x")
    Rl = kl["T"]
    Al = Rl.quo(Zl)
    
    km = k.extension(m, "x")
    Rm = km["T"]
    Am = Rm.quo(Zm)
    
    # We compute the solutions of the Hilbert 90 problems in the first extension of each tower
    
    hl = solve_h90(Al)
    hm = solve_h90(Am)
    
    # We compute a compatible solution in the compositum
    ## l × m
    
    htmplm = solve_h90(Alm)
    alm = Clm([k(x) for x in (htmplm^(l*m)).list()])
    blm = Clm([k(x) for x in (hl^l).list()]).polynomial().subs(Clm.gen()^m)
    clm = Clm([k(x) for x in (hm^m).list()]).polynomial().subs(Clm.gen()^l)
    
    tl_lm = (blm*alm^(-1)).nth_root(l)
    tm_lm = (clm*alm^(-1)).nth_root(m)
    
    HL = Alm(tl_lm.polynomial().list())*htmplm^m
    HM = Alm(tm_lm.polynomial().list())*htmplm^l
    
    hlm = (HL^a)*(HM^b)
    
    return hlm, zlm, zl, zm, HL

In [7]:
def rel_norm(a, k):
    K = a.parent()
    s = k.degree()
    d = K.degree()//s
    return prod([a.frobenius(i*s) for i in range(d)])